# With suppression electrode AND with extraction electrode

Measurement day was 2020-01-23, /Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-01-23.FirstTimeSuppression_withExtraction

In [ ]:
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy as sql
import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns
sns.set()
import getopt
import sys
import datetime
from scipy.interpolate import interp1d
from NGDataObject import NGDataObject
from scipy.optimize import curve_fit
from scipy import optimize

#*******************************
# CONSTANTS
#*******************************
# connection to database
CREDENTIALS_FILE = '/Users/hkromer/02_PhD/01.github/dash_NG/credentials.pw'
DB = "NG_twofast_DB" # name of the database
HOST = "twofast-RPi3-0"  # database host

# LOOKUP TABLES
LUT_PRESSURE_ION_SOURCE = "/Users/hkromer/02_PhD/01.github/phd/01_neutron_generator_contol/LUT_pressure_ion_source.txt"

In [ ]:
DAY = "2020-01-23"
PATH = '/Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-01-23.FirstTimeSuppression_withExtraction/'

# Load from PSI DB

In [ ]:
# reference detectors
# dose
# HV

# read password and user to connect to database
credentials = pd.read_csv(CREDENTIALS_FILE, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]


#*******************************
# GET DATA
#*******************************

# connect to DB
con = NGDataObject(host = HOST, database=DB, user=user, password=pw)

# get dose
query = "SELECT * FROM data_dose WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_dose = con.get_from_database(query=query)

# get HV
query = "SELECT * FROM data_HV WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_hv = con.get_from_database(query=query)
data_hv['HV_current_x100'] = data_hv['HV_current']*100.0

# get pressure
query = "SELECT * FROM data_pressure WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_pressure = con.get_from_database(query=query)

# get microwave power
query = "SELECT * FROM microwave_generator_power WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_microwave_power_full = con.get_from_database(query=query)

# get microwave frequency
query = "SELECT * FROM microwave_generator_frequency WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_microwave_frequency_full = con.get_from_database(query=query)


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_dose.index, y='dose_corrected', data=data_dose, ax=ax, color='darkblue')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('Dose [muSv/hr]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv.index, y='HV_voltage', data=data_hv, ax=ax, color='darkred')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV V [kV]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv.index, y='HV_current', data=data_hv, ax=ax, color='darkorange')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax.set_xlabel(f'Time, {DAY}')
plt.show()


fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_pressure.index, y='pressure_IS_corrected', data=data_pressure, ax=ax, color='darkgreen')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('pressure [mbar]')
ax.set_xlabel(f'Time, {DAY}')
ax.set_yscale('log')
plt.show()


In [ ]:
data_dose.to_csv(f'{PATH}/{DAY}_dose.csv')
data_hv.to_csv(f'{PATH}/{DAY}_hv.csv')
data_pressure.to_csv(f'{PATH}/{DAY}_pressure.csv')
data_microwave_power_full.to_csv(f'{PATH}/{DAY}_microwave_power.csv')
data_microwave_frequency_full.to_csv(f'{PATH}/{DAY}_microwave_frequency.csv')

## Load from csv

In [ ]:
data_dose = pd.read_csv(f'{PATH}/{DAY}_dose.csv', index_col=0)
data_hv = pd.read_csv(f'{PATH}/{DAY}_hv.csv', index_col=0)
data_pressure = pd.read_csv(f'{PATH}/{DAY}_pressure.csv', index_col=0)
data_microwave_power_full = pd.read_csv(f'{PATH}/{DAY}_microwave_power.csv', index_col=0)
data_microwave_frequency_full = pd.read_csv(f'{PATH}/{DAY}_microwave_frequency.csv', index_col=0)

for df in [data_dose, data_hv, data_pressure, data_microwave_power_full, data_microwave_frequency_full]:
    df.index = pd.to_datetime(df.index)

    
time_start = f"{DAY} 11:30:00"
time_end = f"{DAY} 14:05:00"


data_dose = data_dose.loc[time_start:time_end, :]
data_hv = data_hv.loc[time_start:time_end, :]
data_pressure = data_pressure.loc[time_start:time_end, :]
data_microwave_power = data_microwave_power_full.loc[time_start:time_end, :]
data_microwave_frequency = data_microwave_frequency_full.loc[time_start:time_end, :]

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_dose.index, y='dose_corrected', data=data_dose, ax=ax, color='darkblue')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('Dose [muSv/hr]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv.index, y='HV_voltage', data=data_hv, ax=ax, color='darkred')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV V [kV]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv.index, y='HV_current', data=data_hv, ax=ax, color='darkorange')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax.set_xlabel(f'Time, {DAY}')
plt.show()


fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_pressure.index, y='pressure_IS_corrected', data=data_pressure, ax=ax, color='darkgreen')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('pressure [mbar]')
ax.set_xlabel(f'Time, {DAY}')
ax.set_yscale('log')
plt.show()


In [ ]:
def resample_30_seconds(df, cols, range_start, range_end):
    """
    Takes the a grouped df (grouped by day) and resamples the columns cols in 10s
    OUTPUT:
        - dataframe
    """

    d = {}

    s = pd.date_range(start=range_start, end=range_end, freq='10S')
    df_out = pd.DataFrame(pd.Series(s, name='time')).set_index('time')
    for col in cols:
        d[col] = df[col].resample('10S').mean()

    this_d = pd.DataFrame(d)

    df_out = df_out.merge(this_d, left_on=df_out.index, right_on=this_d.index, how='outer')
    df_out = df_out.set_index('key_0')
    df_out.index.name ='time'    
    
    return df_out


In [ ]:
d_hv = resample_30_seconds(data_hv, ['HV_current', 'HV_voltage'], time_start, time_end) 
d_dose = resample_30_seconds(data_dose, ['dose_corrected'], time_start, time_end)

data = pd.merge(d_hv, d_dose, left_index=True, right_index=True, how='outer') # dose is actually not needed
data.head()

In [ ]:
sns.reset_orig()

In [ ]:
# time difference since the start time
data['time_delta_minutes'] = data.index
data['time_delta_minutes'] = data['time_delta_minutes'].apply(lambda x: (x - data.index[0]).seconds/60)


In [ ]:

import os, matplotlib
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
from matplotlib.ticker import AutoMinorLocator
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [ ]:
OUTPUT_FOLDER = '/Users/hkromer/polybox/Thesis/Chapters/FurtherNeutronGeneratorUpgrades/Figures/First_operation_WITH_suppression_with_extraction/'
FIGNAME = 'First_operation_WITH_suppression_with_extraction'

In [ ]:
fig = plt.figure(figsize=(10,6))
fs = 20

plt.plot(data['time_delta_minutes'].values, data['HV_voltage'].values, label='Voltage [-kV]', color='darkred')
plt.plot(data['time_delta_minutes'].values, data['HV_current'].values * 100, label='Current [0.01 mA]', color='darkorange')

ax = plt.gca()

_ = plt.xlabel(r'\textbf{Measurement time [min]}',fontsize=fs)
_ = ax.set_ylabel(r'\textbf{High voltage power supply}', color='black',fontsize=fs)
_ = plt.legend(loc=(0.03, 0.65), fontsize=fs-4)
# minor ticks x
minor_locator = AutoMinorLocator(2)
ax.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax.yaxis.set_minor_locator(minor_locator)
# tick font size
ax.tick_params('x', colors='black', labelsize=fs-2)	
ax.tick_params('y', colors='black', labelsize=fs-2)	
# grid
ax.grid(b=True, which='major', linestyle='-')#, color='gray')
ax.grid(b=True, which='minor', linestyle='--')#, color='gray')


ax2 = ax.twinx()
# Turns off grid on the secondary (right) Axis.
ax2.grid(False)
ax2.set_ylabel(r"\textbf{Ambient dose rate [muSv/hr]}", color='darkblue',fontsize=fs)
ax2.tick_params(axis='y', labelcolor='darkblue', labelsize=fs-2)
plt.plot(data['time_delta_minutes'].values, data['dose_corrected'].values, label='Dose', color='darkblue')

_ = ax2.legend(loc="lower right", fontsize=fs-4)
ax.set_ylim(0, 120)
ax.set_xlim(0, 160)
ax2.set_ylim(0, 480)
ax2.set_yticks(np.arange(0, 480+40, 80))
plt.tight_layout()
plt.savefig(f'{OUTPUT_FOLDER}/{FIGNAME}.pdf')
plt.show()